In [2]:
import os
import sys
import json
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch

In [3]:
device = 'cuda'
print('Use ', torch.cuda.get_device_name(device))

Use  NVIDIA GeForce RTX 2080 Ti


In [4]:
checkpoint = "Salesforce/blip2-opt-2.7b"
dtype = torch.float32

### Config

In [ ]:
from transformers import Blip2Config, Blip2VisionConfig, Blip2QFormerConfig, OPTConfig

configuration = Blip2Config()
print(configuration)
# vision_config = Blip2VisionConfig()
# qformer_config = Blip2QFormerConfig()
# text_config = OPTConfig()

# config = Blip2Config.from_text_vision_config(vision_config, qformer_config, text_config)

### Processor

In [5]:
from transformers import Blip2Processor

processor = Blip2Processor.from_pretrained(
    checkpoint,
    cache_dir='../pretrained_files',
)


In [6]:
print(processor)

Blip2Processor:
- image_processor: BlipImageProcessor {
  "do_convert_rgb": true,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.48145466,
    0.4578275,
    0.40821073
  ],
  "image_processor_type": "BlipImageProcessor",
  "image_std": [
    0.26862954,
    0.26130258,
    0.27577711
  ],
  "processor_class": "Blip2Processor",
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

- tokenizer: GPT2TokenizerFast(name_or_path='Salesforce/blip2-opt-2.7b', vocab_size=50265, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad

In [17]:
print(processor.image_processor)

BlipImageProcessor {
  "do_convert_rgb": true,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.48145466,
    0.4578275,
    0.40821073
  ],
  "image_processor_type": "BlipImageProcessor",
  "image_std": [
    0.26862954,
    0.26130258,
    0.27577711
  ],
  "processor_class": "Blip2Processor",
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}



In [18]:
print(processor.tokenizer)

GPT2TokenizerFast(name_or_path='Salesforce/blip2-opt-2.7b', vocab_size=50265, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True)})


### Model

In [7]:
from transformers import Blip2ForConditionalGeneration

model = Blip2ForConditionalGeneration.from_pretrained(
    checkpoint,
    cache_dir='../pretrained_files',
    torch_dtype=dtype,
    device_map="auto"
)


Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.15s/it]


In [8]:
print(model.config)

Blip2Config {
  "_commit_hash": "56e1fe81e7e7c346e95e196ace7b442b3f8ff483",
  "_name_or_path": "Salesforce/blip2-opt-2.7b",
  "architectures": [
    "Blip2ForConditionalGeneration"
  ],
  "initializer_factor": 1.0,
  "initializer_range": 0.02,
  "model_type": "blip-2",
  "num_query_tokens": 32,
  "qformer_config": {
    "_name_or_path": "",
    "add_cross_attention": false,
    "architectures": null,
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": null,
    "cross_attention_frequency": 2,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_hidden_size": 1408,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": null,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,

In [9]:
print(model)

Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((

In [10]:
print(json.dumps(model.hf_device_map, indent=4))

{
    "query_tokens": 0,
    "vision_model.embeddings": 0,
    "vision_model.encoder.layers.0": 0,
    "vision_model.encoder.layers.1": 0,
    "vision_model.encoder.layers.2": 0,
    "vision_model.encoder.layers.3": 0,
    "vision_model.encoder.layers.4": 0,
    "vision_model.encoder.layers.5": 0,
    "vision_model.encoder.layers.6": 0,
    "vision_model.encoder.layers.7": 0,
    "vision_model.encoder.layers.8": 0,
    "vision_model.encoder.layers.9": 0,
    "vision_model.encoder.layers.10": 0,
    "vision_model.encoder.layers.11": 0,
    "vision_model.encoder.layers.12": 0,
    "vision_model.encoder.layers.13": 0,
    "vision_model.encoder.layers.14": 0,
    "vision_model.encoder.layers.15": 0,
    "vision_model.encoder.layers.16": 0,
    "vision_model.encoder.layers.17": 0,
    "vision_model.encoder.layers.18": 0,
    "vision_model.encoder.layers.19": 0,
    "vision_model.encoder.layers.20": 0,
    "vision_model.encoder.layers.21": 0,
    "vision_model.encoder.layers.22": 0,
    "vis

In [11]:
print(model._modules.keys())

odict_keys(['vision_model', 'qformer', 'language_projection', 'language_model'])


### Vision Model

In [12]:
print(model._modules['vision_model'])

Blip2VisionModel(
  (embeddings): Blip2VisionEmbeddings(
    (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
  )
  (encoder): Blip2Encoder(
    (layers): ModuleList(
      (0-38): 39 x Blip2EncoderLayer(
        (self_attn): Blip2Attention(
          (dropout): Dropout(p=0.0, inplace=False)
          (qkv): Linear(in_features=1408, out_features=4224, bias=True)
          (projection): Linear(in_features=1408, out_features=1408, bias=True)
        )
        (layer_norm1): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
        (mlp): Blip2MLP(
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1408, out_features=6144, bias=True)
          (fc2): Linear(in_features=6144, out_features=1408, bias=True)
        )
        (layer_norm2): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
  (post_layernorm): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
)


### QFormer

In [14]:
print(model._modules['qformer'])

Blip2QFormerModel(
  (layernorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (encoder): Blip2QFormerEncoder(
    (layer): ModuleList(
      (0): Blip2QFormerLayer(
        (attention): Blip2QFormerAttention(
          (attention): Blip2QFormerMultiHeadAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): Blip2QFormerSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (crossattention): Blip2QFormerAttention(
          (attention): Blip2QFormerMultiHeadAttention(
            (query):

### Language Projection

In [15]:
print(model._modules['language_projection'])

Linear(in_features=768, out_features=2560, bias=True)


### Language Model

In [16]:
print(model._modules['language_model'])

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2560, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2560)
      (final_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-31): 32 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (out_proj): Linear(in_features=2560, out_features=2560, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
          (final_layer_norm): Laye